**In this Notebook we aim to build a Traffic sign Classification project using Image Processing and Image Classification**

Their are four phases to accomplish the requirements
- Explore the dataset
- Train the CNN
- Evaluate the CNN

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import tensorflow as tf
from keras.utils import to_categorical
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout


## Explore The Dataset

In [ ]:
#how many classes we have in train
files = []
path = "../input/gtsrb-german-traffic-sign/Train"
for file in os.listdir(path):
    files.append(file)
    
print(len(files))

Train folder contains 43 folder of each class. And in each class it contains various images of that particular class. The range of folder is 0 to 42. With the help of os module, we iterate over all the classes and append images and and their respective labels in data and labels list.

The PIL library is used to open image content into an array.

In [ ]:
from PIL import Image

In [ ]:
from PIL import Image
imgs_path = "../input/gtsrb-german-traffic-sign/Train"
data = []
labels = []
classes = 43

for i in range(43):
    p = os.path.join(imgs_path, str(i)) #0-42
    #print(p)
    for img in os.listdir(p):
        im = Image.open(p + '/' + img)
        im = im.resize((30,30))
        im = np.array(im)
        data.append(im)
        labels.append(i)
        
data = np.array(data)
labels = np.array(labels)
print("success")

In [ ]:
#what we done, see an sample example
i0 = "../input/gtsrb-german-traffic-sign/Train/0/00000_00004_00029.png"
i = Image.open(i0)
i = i.resize((30, 30))
sr = np.array(i)  #it is a matrix of shape (30,30,3)
plt.imshow(i)
plt.show()

In [ ]:
print(data.shape)
print(labels.shape)

Hence we got our data ready, Data is a matrix containing each image of shape 30*30*3 and labels is nothing but a class name from 0 to 42.

In [ ]:
#see any random image
ipath = "../input/gtsrb-german-traffic-sign/Train/7"
imgs = os.listdir(ipath)
for ip in imgs[5:8]:
    im = Image.open(ipath +'/'+ip)
    plt.imshow(im)
    plt.show()

Finally we have sorted all the images and labels into list(data and label). Now we need to convert list into numpy arrays for feeding to the model. The shape of data is (39209, 30, 30, 3) which means we hvae total 39209 images of shape 30*30. And the 3 means data contains colored imgs(RGB)


**Train Test split**

 now we will split the data into train test split. we will one hot encode the labels with to_categorical method imported from keras utils.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("training shape: ",x_train.shape, y_train.shape)
print("testing shape: ",x_test.shape, y_test.shape)

y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

In [ ]:
#after onehot encoding we can see the shape of output labels.
print(y_train.shape, y_test.shape)

## Build CNN Model

It's time for modelling, Concolutional Neural Network is best for Image Classification Task and we will build CNN using Keras Sequential Model.

**The architecture of CNN**

- 1st is Input Convolutional(Conv2D) layer with 32 filters of 5*5 size
- 2nd layer is also conv2d of 32 filters of 5*5
- Next is Max Pooling(MaxPool2D) with 2*2 shape
- Dropout layer(rate=0.25)
- 2 Convolutional Layer of 64 filters of 3*3 size
- MaxPool2D(pool_size=(2,2))
- Dropout layer, rate = 0.25
- Flatter Layer to squeeze the layers in 1-D
- Dense Fully connected layers(256 nodes, activation="relu")
- Dropout(rate=0.5)
- Dense Layer(43 nodes, activation="softmax"), softmax activation for multiclass classification



In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Activation, Flatten, Dropout
from keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation="relu", input_shape=x_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation="softmax"))

* We compile the model using ADAM Optimizer which performs well and loss is "Categorical_crossentropy" because we have multiple classes

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

### Training and Evaluating the model

abhi toh raghav pdna hai, ghar ke bahar car khadi krna hai in an any how condition. I can and I will win.

In [ ]:
epochs = 15
history = model.fit(x_train, y_train, epochs=epochs, batch_size=64, validation_data=(x_test, y_test))

**Let's Plot a graph for accuracy and loss**

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'], label="Training accuracy")
plt.plot(history.history['val_accuracy'], label="val accuracy")
plt.title("Accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend()

plt.figure(1)
plt.plot(history.history['loss'], label="training loss")
plt.plot(history.history['val_loss'], label="val loss")
plt.title("Loss")
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

### Testing the Model

we have test folder which contains different set of images and we also have the respective set of labels. Again we will load all the images and resize them to shape of 30*30*3 And create a Numpy array containing all image data. We will use sklearn accuracy score to check that our model predict or not actual label with 95% accuracy.

In [ ]:
from sklearn.metrics import accuracy_score
import pandas as pd
test = pd.read_csv("../input/gtsrb-german-traffic-sign/Test.csv")
print(test.shape)
test.head()

In [ ]:
test_labels = test['ClassId'].values
test_img_path = "../input/gtsrb-german-traffic-sign"
test_imgs = test['Path'].values

test_data = []
test_labels = []

for img in test_imgs:
    im = Image.open(test_img_path + '/' + img)
    im = im.resize((30,30))
    im = np.array(im)
    test_data.append(im)

test_data = np.array(test_data)
print(test_data.shape)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
test_labels = test['ClassId'].values
test_labels

In [ ]:
#predict the output on test data
predictions = model.predict_classes(test_data)
print("accuracy: ", accuracy_score(test_labels, predictions))

In [ ]:
model.save('traffic_classifier.h5')